In [1]:
import pandas as pd
import numpy as np
import requests
from unidecode import unidecode
import os
import json

import warnings
warnings.filterwarnings('ignore')

In [18]:
# Charger les données

in_file = "../in_data/dossiers_journee-nationale-de-la-resilience-appel-a-projets_2023-08-01_16-59.xlsx"

df_dossier = pd.read_excel(in_file, sheet_name="Dossiers")
df_action_1 = pd.read_excel(in_file, sheet_name="(3344502) Action") # actions ayant lieu qu'une fois
df_action_2 = pd.read_excel(in_file, sheet_name="(3308253) Action") # actions ayant lieu plusieurs fois
df_loc_action2 = pd.read_excel(in_file, sheet_name="(3308215) Localisation")

In [19]:
# Extraire les données nécessaires de l'onglet "Dossiers"

df_dossier = df_dossier[["ID", "Sélectionnez la zone géographique du projet", "Raison sociale", "État du dossier"]].rename(columns={"Raison sociale":"Organisateur", "Sélectionnez la zone géographique du projet": "Zone géographique"})

In [20]:
# Association de l'adresse aux action se déroulant plusieurs fois
# Uniquement possible si une seule action

# On compte le nombre d'actions par dossier
nb_action_2 = df_action_2["Dossier ID"].value_counts().rename("Nb actions").to_frame()
dossier_id_valides = nb_action_2[nb_action_2["Nb actions"] == 1].index.to_list()

# On filtre les dossiers valides
df_action_2 = df_action_2[df_action_2["Dossier ID"].isin(dossier_id_valides)]
df_loc_action2 = df_loc_action2[df_loc_action2["Dossier ID"].isin(dossier_id_valides)]

# On supprime les colonnes en doublon
df_action_2 = df_action_2.drop(columns=["Ligne","Nom de l'action"])

# On fusionne les deux jeux de données pour récupérer les adresses
df_action_2 = pd.merge(df_loc_action2, df_action_2, how="left", on="Dossier ID")

In [28]:
# export des actions dématerialisées (et grand public)
df_action_demat = df_action_1 = df_action_1[
    (~df_action_1["Nom de l'action"].isna()) &
    (df_action_1["L'action est-elle ouverte au grand public ?"] == "Oui") &
    (df_action_1["Votre action est-elle dématérialisée ?"] == "Oui")
]

df_action_demat = pd.merge(
    df_action_demat, 
    df_dossier, 
    how="left", right_on="ID", left_on="Dossier ID"
    )

# Extraction de la zone géographique 
# Ici que pour départements
df_action_demat["Zone géographique"]= pd.Series([l[0] for l in df_action_demat["Zone géographique"].str.split("-")])
df_action_demat["Zone géographique"]

0     92
1     92
2    972
3     17
4    971
5     85
6     93
Name: Zone géographique, dtype: object

In [5]:
# Filtrer les actions :
# - Renseignées
# - Grand public
# - Non dématerialisées

df_action_1 = df_action_1[
    (~df_action_1["Nom de l'action"].isna()) &
    (df_action_1["L'action est-elle ouverte au grand public ?"] == "Oui") &
    (df_action_1["Votre action est-elle dématérialisée ?"] == "Non")
]

df_action_2 = df_action_2[
    (~df_action_2["Nom de l'action"].isna()) &
    (df_action_2["L'action est-elle ouverte au grand public ?"] == "Oui")
]

In [6]:
# Récupération du code de département
df_action_2.loc[df_action_2["Département de l'action"].isna()]["Département de l'action"] = df_action_2[df_action_2["Département de l'action"].isna()]["Commune de l'action (Département)"].str[:2]

In [7]:
# Extraction des colonnes intéressantes
# Changement de nom pour fusionner les deux jeux de données
# Filtrer : action grand public ET non dématérialisée

df_action_1 = df_action_1[[
    "Dossier ID", "Ligne",
    "Nom de l'action", "Description de l'action",
    "Risques ciblés", "Risques naturels", "Risques technologiques",
    "Public cible",
    "Date prévisionnelle du début de l'action", "Date prévisionnelle de la fin de l'action",
    "Veuillez indiquer l'adresse complète de l'action", "Commune", "Département (Code)", "Département"
    ]].rename(columns={
        "Nom de l'action" : "Nom", 
        "Description de l'action" : "Description",
        "Date prévisionnelle du début de l'action" : "Date début", 
        "Date prévisionnelle de la fin de l'action" : "Date fin",
        "Veuillez indiquer l'adresse complète de l'action" : "Adresse", 
        "Commune" : "Nom commune", 
        "Département (Code)" : "INSEE_DEP",
    })

# df_action_1_ = df_action_1_.assign(origine="Action 1")

df_action_2 = df_action_2[[
    "Dossier ID", "Ligne",
    "Nom de l'action", "Description de l'action",
    "Risques ciblés", "Risques naturels", "Risques technologiques",
    "Public cible",
    "Date prévisionnelle du début de l'action", "Date prévisionnelle de la fin de l'action",
    "Adressse complète de l'action", "Commune de l'action", "Département de l'action (Code)", "Département de l'action"
    ]].rename(columns={
        "Nom de l'action" : "Nom", 
        "Description de l'action" : "Description",
        "Date prévisionnelle du début de l'action" : "Date début", 
        "Date prévisionnelle de la fin de l'action" : "Date fin",
        "Adressse complète de l'action" : "Adresse", 
        "Commune de l'action" : "Nom commune", 
        "Département de l'action (Code)" : "INSEE_DEP",
        "Département de l'action" : "Département"
    })

# df_action_2 = df_action_2.assign(origine="Action 2")

# Concatenation des jeux de données
df_action = pd.concat((df_action_1,df_action_2))

# Récupération des informations dossier
df_action = pd.merge(df_action, df_dossier, how="left", left_on="Dossier ID", right_on="ID")

In [8]:
print(len(df_action_1))
print(len(df_action_2))
print(len(df_action))

26
22
48


In [9]:
# Construction des champs True/False

def traitement_champs_multiples(df, col, choix, fillna = False):
    """Conversion des champs à choix multiples en plusieurs champs True / False"""
    
    for c in choix : 
        df["Est "+c] = df.apply(
            lambda x : fillna if not(isinstance(x[col],str)) else (c in x[col].split(", ")),
            axis = 1
        )
        
    return df

In [10]:
for col,choix in [
    ("Risques ciblés", ["Risques naturels","Risques technologiques"]),
    ("Risques naturels", ["Inondations","Feux de forêt","Tempête/cyclone","Séisme","Éruption volcanique","Mouvement de terrain","Risques littoraux","Avalanche","Radon"]),
    ("Risques technologiques", ["Accidents industriels","Accidents nucléaires","Rupture de barrage","Transport de matières dangereuses"]),
    ("Public cible", ["Tous public","Famille","Jeune public","Séniors"]),
] :
    df_action = traitement_champs_multiples(df_action, col, choix)

In [12]:
# Détermination des coordonnées des actions

# Récolte des coordonnées et département pour actions

def adress2latlon(adress, commune, dpt):
    
    """Interroge l'API de recherche d'adresse de OpenStreetMap
    - Si adresse non rentrée par le dépositaire du dossier OU non trouvée par OpenStreetMap : on cherche les coordonnées de la commune
    - Sinon on donne les coordonnées de l'adresse précise
    """
    
    if not(isinstance(adress, str)):
        return [np.nan,np.nan]
    
    url = f"https://nominatim.openstreetmap.org/?adressdetails=1&q={adress.replace(' ','+')}&format=json&limit=1"
    
    try :
        
        data = requests.get(url).json()[0]
                
        # commune = data["display_name"].split(", ")[2]
        # dpt = data["display_name"].split(", ")[4]

        return [data["lat"],data["lon"]]
    
    except :
        print(f"Warning : L'adresse suivante n'a pas été trouvée : {adress}, on donne les coordonnées de la commune")
        
        url = f"https://nominatim.openstreetmap.org/?adressdetails=1&q={commune.replace(' ','+')},+{dpt}&format=json&limit=1"

        try : 
            data = requests.get(url).json()[0]
            return [data["lat"],data["lon"]]
        
        except :
            print(f"Warning : Commune {commune} ({dpt}) non trouvée")
            return [np.nan,np.nan]
            
df_action[['Lat', 'Lon']] = pd.DataFrame([adress2latlon(adress,com,dpt) for adress,com,dpt in zip(df_action["Adresse"],df_action["Nom commune"],df_action["Département"])],index=df_action.index)   

In [18]:
# # On filtre les actions dont les coordonnées n'ont pas été trouvée

print(f"Warning : les coordonnées de {df_action['Lat'].isna().sum()} actions n'ont pas pu être déterminées.")
df_action = df_action[~df_action['Lat'].isna()]


In [20]:
# Export des données pour la carte

df_action = df_action.assign(
    action_id = np.nan
    )

df_action["action_id"] = df_action.apply(
    lambda r : f'{r["Dossier ID"]}_{r["Ligne"]:02d}', axis=1
)

df_action = df_action.drop(columns=["Ligne"])

df_action = df_action.rename(columns=dict(zip(df_action.columns, [unidecode(c).lower().replace(" ","_").replace("/","_") for c in df_action.columns])))

df_action.to_csv(
    "../out_data/actions_grand_public_carte.csv", index=False
)

df_action.to_csv(
    "../out_data/actions_grand_public_carte_ansi.csv", index=False, sep=";", encoding="ansi"
)

KeyError: 'Dossier ID'

In [21]:
df_action.to_csv(
    "../out_data/actions_grand_public_carte.csv", index=False
)

df_action.to_csv(
    "../out_data/actions_grand_public_carte_ansi.csv", index=False, sep=";", encoding="ansi"
)

In [34]:
f"{int(1.):02d}"

'01'

,dossier_id,nom,description,risques_cibles,risques_naturels,risques_technologiques,public_cible,date_debut,date_fin,adresse,...,est_accidents_nucleaires,est_rupture_de_barrage,est_transport_de_matieres_dangereuses,est_tous_public,est_famille,est_jeune_public,est_seniors,lat,lon,action_id
29,13002567,Ateliers éducatifs autour des étangs,Informer sur la prise en compte du risque de r...,Risques naturels,"Inondations, Feux de forêt",NaN,Tous public,2023-10-09,2023-10-13,route du Lac 19300 Égletons,...,False,False,False,True,False,False,False,45.4168778,2.0584588,NaN


In [37]:
# Compilation des données au niveau départemental 

# On retire les actions pour lesquelles le département n'est pas indiqué
df_action = df_action[~df_action["insee_dep"].isna()]

# On s'assure de convertir les codes de département en chaînes de caractère valables
if df_action["insee_dep"].dtype == float:
    df_action["insee_dep"] = df_action.apply(lambda r : f"{int(r['insee_dep']):02d}", axis=1)

# On retient que les actions ouvertes au grand public 

risques = ["Inondations","Feux de forêt","Tempête/cyclone","Séisme","Éruption volcanique","Mouvement de terrain","Risques littoraux","Avalanche","Radon","Accidents industriels","Accidents nucléaires","Rupture de barrage","Transport de matières dangereuses"]
col_risques = [f"est_{unidecode(c).lower().replace(' ','_').replace('/','_')}" for c in risques]

df_nb_actions_risque = df_action.groupby("insee_dep")[col_risques].sum()
df_nb_actions = df_action["insee_dep"].value_counts()

nb_actions_risque_dict = {}

couleurs_risques = {
    "Inondations" : "#74BFDF",
    "Feux de forêt" : "#FC7C7C",
    "Tempête/cyclone" : "#B4CAD3",
    "Séisme" : "#D3C7B4",
    "Éruption volcanique" : "#FFB475",
    "Mouvement de terrain" : "#A87C56",
    "Risques littoraux" : "#A9FBF7",
    "Avalanche" : "#E4E6E6",
    "Radon" : "#FDC7FD",
    "Accidents industriels" : "#B862F5",
    "Accidents nucléaires" : "#F2ED5A",
    "Rupture de barrage" : "#4046F5",
    "Transport de matières dangereuses" : "#F75CA9"
}

couleurs = [couleurs_risques[r] for r in risques]

# Données INSEE
insee_data_dir = "../in_data"
df_dpt = pd.read_csv(os.path.join(insee_data_dir, "DEPARTEMENT.csv"))

for code in df_dpt["INSEE_DEP"]:
    nb_actions_risque_dict[code] = {
        "Nombre actions" : 0
    }

for code in df_nb_actions.index:
    try : 
        nb_actions_risque_dict[code]["Nombre actions"] = df_nb_actions.loc[code].item()
    except KeyError:
        print(f"Entrées régionales non prises en compte pour le moment (région {code})")

for code in df_nb_actions_risque.index:
    try :
        data = df_nb_actions_risque.loc[code].values.flatten()
        data_dpt = nb_actions_risque_dict[code]
        data_dpt["Type de risque"] = np.array(risques)[data!=0].tolist()
        data_dpt["Nombre d'actions"] = data[data!=0].tolist()
        data_dpt["couleur_plot"] = np.array(couleurs)[data!=0].tolist()
    except KeyError:
        print(f"Entrées régionales non prises en compte pour le moment (région {code})")


In [40]:
with open('../out_data/nb_actions_dpt_08_07_new_names.json', 'w') as fp:
    json.dump(nb_actions_risque_dict, fp, indent=4)

In [48]:
df1 = pd.DataFrame.from_dict({"A":[1,2.],"B":["a","b"]})
df2 = pd.DataFrame.from_dict({"A":[1,2.,3],"C":["A","B","C"], "D":["A","B","C"]})

df3 = pd.merge(df1, df2, how="left", on="A")


In [49]:
df3

,A,B,C,D
0,1.0,a,A,A
1,2.0,b,B,B
